In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

titanic = sns.load_dataset('titanic')

In [2]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#1. 성별과 생존별 관계 파악

In [3]:
titanic.groupby(['sex','survived'])[['survived']].count()

survived
sex    survived          
female 0               81
       1              233
male   0              468
       1              109

여성이 남성보다 생존률이 더 높다.

#2. 나이대별 파생변수 추가

In [4]:
# age의 최솟값, 최댓값 구한 뒤 범위 나누기 
# min : 0.42, max : 80
titanic['age'].agg(['min','max'])

# 나이대별 age 그룹 나누기
# 0~20 -> child
# 21~40 -> young
# 41~60 -> adult
# 61~80 -> older

titanic['age_group'] = pd.cut(titanic['age'],bins=[0,20,40,60,80], labels = ['child','young','audlt','older'])
titanic['age_group'].value_counts()

young    385
child    179
audlt    128
older     22
Name: age_group, dtype: int64

#3. 나이대별 생존 관계 파악

In [5]:
titanic.groupby(['age_group','survived']).agg(n = ('survived','count'))

n
age_group survived     
child     0          97
          1          82
young     0         232
          1         153
audlt     0          78
          1          50
older     0          17
          1           5

생존한 그룹만 봤을 때 young의 그룹이 제일 많이 생존했고, older 그룹이 제일 적게 생존했다.

#4. 'class'별 생존율은?

In [6]:
titanic.groupby(['class','survived']).agg(n=('survived','count'))

n
class  survived     
First  0          80
       1         136
Second 0          97
       1          87
Third  0         372
       1         119

class가 좋을 수록 생존할 확률이 더 높다.

#5. SibSp + Parch를 더한 Family 파생변수 만들기

In [7]:
titanic1 = titanic.assign(family = titanic['sibsp']+titanic['parch'])
titanic1[['survived','family']]

,survived,family
0,0,1
1,1,1
2,1,0
3,1,1
4,0,0
...,...,...
886,0,0
887,1,0
888,0,3
889,1,0


#6. 가족 존재 유무에 따른 생존률 비교

In [8]:
titanic2 = titanic1.assign(family_yn = np.where(titanic1['family']>0,'Y','N'))[['survived','family','family_yn']]
titanic2

,survived,family,family_yn
0,0,1,Y
1,1,1,Y
2,1,0,N
3,1,1,Y
4,0,0,N
...,...,...,...
886,0,0,N
887,1,0,N
888,0,3,Y
889,1,0,N


In [9]:
titanic2.groupby(['family_yn','survived']).agg(n=('survived','count'))

n
family_yn survived     
N         0         374
          1         163
Y         0         175
          1         179

가족이 없는 경우 생존확률이 더 적었고, 가족이 있는 경우는 생존 확률과 사망 확률이 비슷하다.

#7. 클래스별 평균 요금은?

In [10]:
titanic.groupby('class').mean()[['fare']]

,fare
class,
First,84.154687
Second,20.662183
Third,13.675550


#8. embark_town에 따른 class별 탑승수

In [11]:
titanic.groupby(['embark_town','class']).agg(n=('class','count'))

n
embark_town class      
Cherbourg   First    85
            Second   17
            Third    66
Queenstown  First     2
            Second    3
            Third    72
Southampton First   127
            Second  164
            Third   353

Cherbourg에서는 First class, Queenstown에서는 Third class, Southampton에서는 Third class에 가장 많은 수가 탑승했다.

#9. embarked와 class의 생존관계 비교

In [12]:
titanic.query('survived==1').groupby(['class','embarked','survived']).agg(n=('survived','count'))

n
class  embarked survived    
First  C        1         59
       Q        1          1
       S        1         74
Second C        1          9
       Q        1          2
       S        1         76
Third  C        1         25
       Q        1         27
       S        1         67

First class, Second class,Third class 모두  Southampton에서 탑승한 승객의 생존률이 더 높다.

#10. class별 남녀의 생존확률은?

In [16]:
titanic.query('survived==1').groupby(['class','sex']).agg(n=('survived','count'))

n
class  sex       
First  female  91
       male    45
Second female  70
       male    17
Third  female  72
       male    47

class 모두 여성의 생존확률이 남성보다 더 높다.